In [ ]:
import polars as pl
import validate.lesion as vl
import validate.general as vg

from constants import list_id_cols

import utils
from pathlib import Path
import os

PATH_INPUT = os.getenv("PATH_INPUT")

lf = utils.get_df(Path(PATH_INPUT).joinpath("test data.accdb")).lazy()

lf2 = utils.get_df(Path(PATH_INPUT).joinpath("ZON CHERAS OC DATABASE JULY-OCT 2023 edit MAIwani.accdb")).lazy()

obj = vg.ValidationGeneral(lf, "test data")
lfall = pl.concat([lf, lf2])

In [ ]:
lfall.columns

In [ ]:
intervention_status_whitelist = [
  "Null|Null",
  "TIDAK HADIR|II- GAGAL DATANG TEMUJANJI",
  "HADIR|I- SEDANG MENERIMA RAWATAN",
  "HADIR|III- GAGAL BERHENTI",
  "HADIR|IV- BERJAYA BERHENTI SELAMA 6 BULAN",
]

lfall.select("HADIR QUIT SERVICES", "STATUS INTERVENSI").select(
  pl.col("HADIR QUIT SERVICES", "STATUS INTERVENSI").fill_null("Null")
).with_columns(
  pl.concat_str(
    pl.col("HADIR QUIT SERVICES", "STATUS INTERVENSI"), separator=("|")
  ).alias("concat")
).filter(~pl.col("concat").is_in(intervention_status_whitelist)).collect()
